In [1]:
# Dependencies
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pprint import pprint
from sodapy import Socrata




In [2]:
client = Socrata("data.iowa.gov", None)
results = client.get("ykb6-ywnd", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
#results_df.head()

In [3]:
new_df=results_df[["store_address_address", "city","store_status","state", "store_address_zip"]]
new_df.columns = ["Address", "City", "Status","State", "Zip Code"]
active_stores_df=new_df.loc[new_df["Status"]=="A"].reset_index(drop=True)
active_stores_df.head()

,Address,City,Status,State,Zip Code
0,200 State Po Box 98,Guthrie Center,A,IA,50115
1,1111 8TH ST,Boone,A,IA,50036
2,3140 Agency,Burlington,A,IA,52601
3,939 Angular,Burlington,A,IA,52601
4,"1843 Johnson Avenue, N.W.",Cedar Rapids,A,IA,52405


In [4]:
active_stores_df['lat'] = ""
active_stores_df['lng'] = ""

params = {"key":gkey}
for index, row in active_stores_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    street_type = row['Address']
    city_type = row['City']
    state_type = row['State']
    params['address'] = f"{street_type}{city_type},{state_type}"
    store_lat_lng = requests.get(base_url, params=params).json()
    active_stores_df.loc[index, "lat"] = store_lat_lng["results"][0]["geometry"]["location"]["lat"]
    active_stores_df.loc[index, "lng"] = store_lat_lng["results"][0]["geometry"]["location"]["lng"]
active_stores_df.head()

,Address,City,Status,State,Zip Code,lat,lng
0,200 State Po Box 98,Guthrie Center,A,IA,50115,41.6774,-94.5026
1,1111 8TH ST,Boone,A,IA,50036,42.0647,-93.8761
2,3140 Agency,Burlington,A,IA,52601,40.8162,-91.1367
3,939 Angular,Burlington,A,IA,52601,40.8013,-91.1106
4,"1843 Johnson Avenue, N.W.",Cedar Rapids,A,IA,52405,41.9739,-91.6984


In [5]:
active_stores_df.to_csv('liquor_store_ia.csv')

Figure(layout=FigureLayout(height='420px'))